## 1.Building the Dataset

To build a Viterbi Decoder , we need to access a trained corpora. This will help us to calculate the emission and transition probabilities. After importing the relevant libraries, we import the treebank corpora. This is a set of Wall Street Journal Articles.

In [3]:
import nltk
import random, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import metrics

nltk.download('treebank')
nltk.download('universal_tagset')

wsj = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


Once we split the dataset, we can see how many sentences are there in the training and test set. We can also see, how many words are there in each of them

In [4]:
#random.seed(0)
train_set,test_set=train_test_split(wsj,test_size=0.05,random_state=10)

print('length of train set {}'.format(len(train_set)))
print('length of test set {}'.format(len(test_set)))

train_tagged_words=[tup for sent in train_set for tup in sent]
test_tagged_words=[tup for sent in test_set for tup in sent]

print('Number of tagged train words {}'.format(len(train_tagged_words)))
print('Number of tagged test words {}'.format(len(test_tagged_words)))

length of train set 3718
length of test set 196
Number of tagged train words 95440
Number of tagged test words 5236


## 2.Emission and Transition Probabilities

In order to build the Viterbi decoder , we have to calculate the:

- Transition Probability for each tag combination. This is the matrix __trans__.
- In theory , we could make a matrix for Emission Probability as well, for each tag-word combination.However, there's a reason we won't.




In [5]:
# Building the vocabulary and list of tags
vocab=list(set([word.lower() for word,pos in train_tagged_words]))
tags=list(set([pos for word,pos in train_tagged_words]))

# Count of unique words and tags
all_words=len(vocab)
all_tags=len(tags)

print('Number of words in Vocabulary {}'.format(all_words))
print('Number of tags {}'.format(all_tags))

"""
trans is a matrix all tags vs all tags. An element in this matrix gives
the probability of a tag (row), given a previous tag ( column ). We set initialize this matrix to 0's
"""

trans=np.zeros((all_tags,all_tags))

Number of words in Vocabulary 11017
Number of tags 12


To build a matrix of emission probability , we need two nested loops to reach all the words in the vocab and all the tags. With our vocabulary size of 11017 and 12 tags, this would mean more than 100K iterations. There's a better way to handle that.<br> Note that, we need to calculate the $P(w|t)$ of only those words that we'll encounter in the test set. So, when we traverse the words in the test set, we can calculate the $P(w|t)$ on the fly.<br>

We define two functions that will help populate the two matrices we defined above
- __word_tag__ generates the count of a word associated with a tag and how many times that tag occured.
- __t2_given_t1__ genrates the count of a certain tag t2 occurred after a tag t1.

In [6]:
def word_tag(this_word,this_tag,tagged_words=train_tagged_words):

  #List of tuples where tag matches this_tag
  this_word=this_word.lower()
  tuples_this_tag=[(word,tag) for word,tag in tagged_words if tag==this_tag]

  #List of tuples from tuples_this_tag where word is this_word
  word_given_tag=[word for word,tag in tuples_this_tag if word.lower()==this_word]

  #Count of this_word passed to the function with this_tag
  #and count of total number of times, this_tag occured  
  
  tag_count=len(tuples_this_tag)
  word_count=len(word_given_tag)
  
  return (word_count,tag_count)

def t2_given_t1(t1,t2,tagged_words=train_tagged_words):
  
  # How many times tag t1 occured in the training set
  this_tag=[tag for word,tag in tagged_words if tag==t1]
  tag_count=len(this_tag)

  # How many time tag t1 was followed by tag t2
  t2_count=0
  for n in range(len(tagged_words)-1):
    if tagged_words[n][1]==t1 and tagged_words[n+1][1]==t2:
      t2_count+=1

  return (t2_count,tag_count)

Let us try both the functions for some example combination. The following code snippet shows that the word __Your__ occured as a pronoun 27 times and there are 2588 Pronouns overall in the training set. Similarly the tag __DET__ was followed by __NOUN__ 5298 times out of 8301 determinants in the training set.

In [7]:
print(word_tag('Your','PRON'))
print(t2_given_t1('DET','NOUN'))

(26, 2604)
(5286, 8287)


We can use these numbers to calculate the following emission and transition probabilities.<br>
$P('Your'|'PRON')=27/2604=0.010368$<br>
$P('PRON'|'DET')=5286/8287=0.637866$<br>

We can calculate the emission matrix __emsn__ for each possible tag transition. The following code generates a dataframe __tags_df__. Each row in the dataframe is t2 and column is t1. For example, to calculate the probability of __NOUN__ , given the previous tag was __DET__, we locate the __NOUN__ row and __DET__ column.

In [8]:
for ro,t2 in enumerate(tags):
  for col,t1 in enumerate(tags):
    trans[ro,col]=t2_given_t1(t1,t2)[0]/t2_given_t1(t1,t2)[1]

tags_df = pd.DataFrame(trans, columns = tags, index=tags)
tags_df

,NOUN,CONJ,PRT,.,VERB,X,ADV,PRON,NUM,ADJ,DET,ADP
NOUN,0.263653,0.350746,0.249674,0.217777,0.110515,0.060863,0.031520,0.209293,0.355193,0.699393,0.637867,0.320990
CONJ,0.042643,0.000466,0.001958,0.058110,0.005207,0.010064,0.007299,0.005376,0.013650,0.017403,0.000483,0.000854
PRT,0.044546,0.004664,0.001958,0.002444,0.031476,0.184824,0.014599,0.012289,0.025816,0.010671,0.000241,0.001281
.,0.238470,0.035914,0.042428,0.093682,0.035362,0.162780,0.137027,0.039171,0.118101,0.064029,0.017377,0.039484
VERB,0.146889,0.155317,0.399478,0.088975,0.169193,0.206709,0.345720,0.485407,0.017804,0.011821,0.039218,0.008537
X,0.028807,0.008396,0.012728,0.027697,0.217689,0.074121,0.021566,0.092550,0.208309,0.021015,0.046096,0.034895
ADV,0.016947,0.055037,0.010444,0.054218,0.081293,0.026038,0.078301,0.034562,0.003264,0.004761,0.012188,0.013446
PRON,0.004795,0.059701,0.017624,0.066618,0.036294,0.054952,0.014930,0.007680,0.001187,0.000493,0.003258,0.068829
NUM,0.009810,0.041511,0.054830,0.081644,0.022072,0.002875,0.032847,0.006912,0.184866,0.020850,0.021841,0.063280
ADJ,0.012152,0.113340,0.086162,0.045076,0.064972,0.016773,0.128733,0.073733,0.032938,0.066820,0.206951,0.106712


## 3.Viterbi Decoder

This is the most important part of our Viteri Algorithm. The inner working of the function is as follows.

- Pass a list of words from a sentence to the function.
- For each word $w$ in the sentence , calculate  $P(w|t_i)*P(t_i|t_{i-1})$ for each of the tags.
- Select the index of the maximum product and use it to point at the __tags__ list to predict a tag for the word $w$.
- The predicted tag is stored in a list __tag_seq__. This is used to fetch the previous tag, unless we are the beginning of a sentence.
- If , we are the beginning of a sentence, the previous tag is "." (start tag).
- The function returns a list of tuple, with each tuple containing the word and the predicted tag.
- In case , we do not find a word in the vocabulary, we simply use a tag with maximum transition probability.


In [9]:
def Viterbi(words,vocab=vocab,tags_df=tags_df):
  #An empty list to hold the predicted tags
  tag_seq=[]

  #Run through each tuple in the sentence. Each tuple has a word and a tag
  for n,word in enumerate(words):

  #Initialize a list to hold the product of emission and transition
  #probabilities 

    argmax=[]
    if word in vocab:
  #For each of the tags we calculate the product of transition and emission probabilities
  # and append it to argmax 
      for tag in tags:
        if n==0:
          #transition probability
          trans_p=tags_df.loc[tag,'.']
        else:
          trans_p=tags_df.loc[tag,tag_seq[n-1]]

        emisn_p=word_tag(word,tag)[0]/word_tag(word,tag)[1]
        argmax.append(trans_p*emisn_p)

  #argmax has now a product for each tag. Select the index of the maximum product
  #This index is the predicted tag for current word. 
      max_p=max(argmax)
      max_p_tag_idx=argmax.index(max_p)
      tag_seq.append(tags[max_p_tag_idx])
    
  #If this word from the test set is not in the Vocabulary, then we just use the transition 
  #probability to predict a tag.
    else:
      if len(tag_seq)==0:
        prev_tag='.'
      else:
        prev_tag=tag_seq[n-1]
      
      argmax=list(tags_df.loc[::,'.'].values)
      max_p=max(argmax)
      max_p_tag_idx=argmax.index(max_p)
      tag_seq.append(tags[max_p_tag_idx])
  
  return list(zip(words,tag_seq))

## 4.Prediction and Results

Now, we simply take the test set and strip the true tags from each word. We pass the resulting list of words to the Viterbi function and store the predictions in __pred_set__   

In [14]:
pred_set=[Viterbi([word for word,pos in sent]) for sent in test_set]

The performance of the Viterbi Algorithm can be assessed by comparing __pred_set__ and __test_set__

In [15]:
pred_tags=[[pos for word,pos in sent] for sent in pred_set]
true_tags=[[pos for word,pos in sent] for sent in test_set]

print(metrics.flat_classification_report(true_tags, pred_tags, labels=tags, digits=2))

              precision    recall  f1-score   support

        NOUN       0.75      0.98      0.85      1547
        CONJ       0.99      0.87      0.93       121
         PRT       0.99      0.97      0.98       155
           .       1.00      0.99      0.99       667
        VERB       0.96      0.87      0.91       697
           X       1.00      0.56      0.72       353
         ADV       0.89      0.80      0.84       157
        PRON       1.00      0.92      0.96       133
         NUM       0.99      0.82      0.89       176
         ADJ       0.88      0.64      0.74       306
         DET       0.95      0.84      0.89       438
         ADP       0.98      0.88      0.93       486

    accuracy                           0.88      5236
   macro avg       0.95      0.84      0.89      5236
weighted avg       0.90      0.88      0.88      5236

